## Initialize all imports

In [1]:
import os
import conda_installer
os.environ["CUDA_VISIBLE_DEVICES"] = "3"  # Must come BEFORE importing TensorFlow
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/opt/nvidia/cuda/cuda-11.6'
import pandas as pd
import tensorflow as tf
import numpy as np
from rdkit import Chem
from deepchem.feat.graph_features import atom_features as get_atom_features
import rdkit
import pickle
import copy
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


import importlib
import keras.backend as K
# import tensorflow_addons as tfa
from tensorflow.keras import regularizers, constraints, callbacks

import sys
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam


2025-04-28 15:49:13.942172: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-28 15:49:14.010323: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normaliz

## Read Input data & Change directory

In [2]:
df = pd.read_csv('Datasets/pdbbind_100.csv')
PDBs = pickle.load(open('Datasets/PDBBind_100.pkl', 'rb'))


In [3]:
df = df[df['complex-name'] != '1.00E+66']


In [4]:
len(df)

98

#### PDB PICKLE FILE GENERATION FOR 100 molecule [NOT NEEDED]

In [5]:

# complex_names_df = df['complex-name'].to_numpy()
# PDBs = {}
# from os import listdir
# from os.path import isfile, join
# mypath = 'pdbbind_complex'
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# for f in onlyfiles:
#     if f.split('.')[0] in complex_names_df:
#         PDBs.update({f.split('.')[0] : rdkit.Chem.rdmolfiles.MolFromPDBFile(mypath + '/' + f)})
        
# PDBs = {pdb: value for pdb, value in PDBs.items() if value is not None}
# pkl_file = open('PDBBind_100.pkl', 'wb')
# pickle.dump(PDBs, pkl_file)
# pkl_file.close()

## DATA PREPROCESSING df ~ pdb

In [6]:
pdb_keys = set(PDBs.keys())
df_filtered = df[df['complex-name'].isin(pdb_keys)]
len(df_filtered)

98

In [7]:
df_final = df_filtered[['complex-name','pb-protein-vdwaals', 'pb-ligand-vdwaals', 'pb-complex-vdwaals', 'gb-protein-1-4-eel', 'gb-ligand-1-4-eel', 'gb-complex-1-4-eel',
       'gb-protein-eelect', 'gb-ligand-eelec', 'gb-complex-eelec', 'gb-protein-egb', 'gb-ligand-egb', 'gb-complex-egb', 'gb-protein-esurf', 'gb-ligand-esurf', 'gb-complex-esurf','ddg']]

In [8]:
len(df_final)

98

## Data pre-processing

In [9]:
import models.layers_update_mobley as layers
from models.dcFeaturizer import atom_features as get_atom_features
importlib.reload(layers)

<module 'models.layers_update_mobley' from '/home/lthoma21/BFE-Loss-Function/FINAL-PDBBIND-FILES/models/layers_update_mobley.py'>

In [10]:
info = []
for pdb in list(PDBs.keys()):
    info.append(df_final[df_final['complex-name'] == pdb][['pb-protein-vdwaals', 'pb-ligand-vdwaals', 'pb-complex-vdwaals', 'gb-protein-1-4-eel', 'gb-ligand-1-4-eel', 'gb-complex-1-4-eel',
       'gb-protein-eelect', 'gb-ligand-eelec', 'gb-complex-eelec', 'gb-protein-egb', 'gb-ligand-egb', 'gb-complex-egb', 'gb-protein-esurf', 'gb-ligand-esurf', 'gb-complex-esurf']].to_numpy()[0])

In [11]:
def featurize(molecule, info):
    
    atom_features = []
    for atom in molecule.GetAtoms():
        new_feature = get_atom_features(atom).tolist()
        position = molecule.GetConformer().GetAtomPosition(atom.GetIdx())
        new_feature += [atom.GetMass(), atom.GetAtomicNum(),atom.GetFormalCharge()]
        new_feature += [position.x, position.y, position.z]
        for neighbor in atom.GetNeighbors()[:2]:
            neighbor_idx = neighbor.GetIdx()
            new_feature += [neighbor_idx]
        for i in range(2 - len(atom.GetNeighbors())):
            new_feature += [-1]

        atom_features.append(np.concatenate([new_feature, info], 0))
    return np.array(atom_features)

In [12]:
X = []
y = []
for i, pdb in enumerate(list(PDBs.keys())):
    X.append(featurize(PDBs[pdb], info[i]))
    y.append(df_final[df_final['complex-name'] == pdb]['ddg'].to_numpy()[0])


In [13]:
#  To find max number of atoms in the Dataframe after featurizing
# Required to provide max_atom_number for padding the X_train
# x = []
# y=[]
# for i in range(len(X)):
#     shape = X[i].shape
#     x.append(shape[0])
#     y.append(shape[1])
# max(x)


In [14]:
# Split the data into training and testing sets
# Randomly shuffles the data before splitting, ensuring that the training and testing sets are representative of the overall dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)


In [15]:
len(X_train), len(X_test), len(y_train), len(y_test)

(78, 20, 78, 20)

## Helper Functions

In [16]:

class PGGCNModel(tf.keras.Model):
    def __init__(self, num_atom_features=36, r_out_channel=20, c_out_channel=1024, l2=1e-2, dropout_rate=0.4, maxnorm=3.0):
        super().__init__()
        self.ruleGraphConvLayer = layers.RuleGraphConvLayer(r_out_channel, num_atom_features, 0)
        self.ruleGraphConvLayer.combination_rules = []
        self.conv = layers.ConvLayer(c_out_channel, r_out_channel)
        self.dense1 = tf.keras.layers.Dense(32, activation='relu', name='dense1', kernel_regularizer=regularizers.l2(l2), bias_regularizer=regularizers.l2(l2), kernel_constraint=constraints.MaxNorm(maxnorm))
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dense5 = tf.keras.layers.Dense(16, activation='relu', name='dense2', kernel_regularizer=regularizers.l2(l2), bias_regularizer=regularizers.l2(l2), kernel_constraint=constraints.MaxNorm(maxnorm))
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)
        self.dense6 = tf.keras.layers.Dense(1, name='dense6', kernel_regularizer=regularizers.l2(l2), bias_regularizer=regularizers.l2(l2), kernel_constraint=constraints.MaxNorm(maxnorm))
        self.dense7 = tf.keras.layers.Dense(1, name='dense7',
                                             kernel_initializer=tf.keras.initializers.Constant([0.3, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1]),
                                             bias_initializer=tf.keras.initializers.Zeros(),
                                             kernel_regularizer=regularizers.l2(l2), 
                                             bias_regularizer=regularizers.l2(l2), 
                                             kernel_constraint=constraints.MaxNorm(maxnorm))

    def addRule(self, rule, start_index, end_index=None):
        self.ruleGraphConvLayer.addRule(rule, start_index, end_index)
        
    def set_input_shapes(self, i_s):
        self.i_s = i_s

    def call(self, inputs, training=False):
        print("Inside call")
        physics_info = inputs[:, 0, 38:] 
        x_a = []
        for i in range(len(self.i_s)):
            x_a.append(inputs[i][:self.i_s[i], :38])
        x = self.ruleGraphConvLayer(x_a)
        x = self.conv(x)
        x = self.dense1(x)
        x = self.dropout1(x, training=training)
        x = self.dense5(x)
        x = self.dropout2(x, training=training)
        model_var = self.dense6(x)
        merged = tf.concat([model_var, physics_info], axis=1)
        out = self.dense7(merged)
        return tf.concat([out, physics_info], axis=1)
    
class LossComponentsCallback(tf.keras.callbacks.Callback):
    def __init__(self,model_instance):
        super().__init__()
        self.empirical_losses = []
        self.physical_losses = []
        self.total_losses = []
        self.learning_rates = []
        self.model = model_instance
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # Store the total loss
        self.total_losses.append(logs.get('loss'))
        lr = self.model.optimizer.learning_rate
        if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule):
            lr = lr(self.model.optimizer.iterations)  # Call the schedule
        else:
            lr = lr  

        self.learning_rates.append(float(tf.keras.backend.get_value(lr)))


def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred[0] - y_true))) + K.abs(1 / K.mean(.2 + y_pred[1]))


def pure_rmse(y_true, y_pred):
    y_true_flat = tf.reshape(y_true, [-1])
    return K.sqrt(K.mean(K.square(y_pred - y_true_flat)))

def physical_consistency_loss(y_true,y_pred,physics_info):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    dG_pred = y_pred
    y_true = tf.reshape(y_true, (-1, 1))

    # Physical Inconsistency loss
    # Extract the components from physics_info
    host = tf.gather(physics_info, [0, 3, 6, 9, 12], axis=1)  # Host energy terms
    guest = tf.gather(physics_info, [1, 4, 7, 10, 13], axis=1)  # Guest energy terms
    complex_ = tf.gather(physics_info, [2, 5, 8, 11, 14], axis=1)  # Complex energy terms

    # Calculate ΔG based on physics: ΔG = ΔGcomplex - (ΔGhost + ΔGguest)
    dG_physics = tf.reduce_sum(complex_, axis=1, keepdims=True) - (tf.reduce_sum(host, axis=1, keepdims=True) + tf.reduce_sum(guest, axis=1, keepdims=True))
    phy_loss = K.sqrt(K.mean(K.square(dG_pred - dG_physics)))
#     tf.print("dG_physics:", dG_physics)
#     tf.print("dG_pred:", dG_pred)
    return phy_loss



def combined_loss(physics_hyperparam=0.0003):
    def loss_function(y_true, y_pred):
        # Extract prediction and physics info
#         y_true = tf.cast(y_true, tf.float32)
#         y_pred = tf.cast(y_pred, tf.float32)
        prediction = y_pred[:, 0]
        physics_info = y_pred[:, 1:16]  # Assuming 15 physical features
        
        # Calculate individual loss components
        empirical_loss = pure_rmse(y_true, prediction)
        physics_loss = physical_consistency_loss(y_true, prediction, physics_info)
        
        # Combine losses with weights
        total_loss = empirical_loss + (physics_hyperparam * physics_loss)
        
        return total_loss
    
    return loss_function






In [17]:
X_train[0].shape

(4193, 53)

In [ ]:
import csv
import time

physics_hyperparam = [0.000002]
epochs = [100]
lr_schedule = ExponentialDecay(
        initial_learning_rate=0.0001,
        decay_steps=5000,
        decay_rate=0.85,
        staircase=True
    )

y_differences = []
total_losses = []
empirical_losses = []
physics_losses = []
all_results=[]
start = time.time()   
for epoch in epochs:
        for physics_weight in physics_hyperparam:
            print("---------- Hyperparameter combinations ------------")
            print("Epoch : {};  physics_weight: {};".format(str(epoch),  str(physics_weight)))

            m = PGGCNModel()
            m.addRule("sum", 0, 32)
            m.addRule("multiply", 32, 33)
            m.addRule("distance", 33, 36)

            opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
            m.compile(loss=combined_loss(physics_weight), optimizer=opt)

            input_shapes = []
            for i in range(len(X_train)):
                input_shapes.append(np.array(X_train[i]).shape[0])
            m.set_input_shapes(input_shapes)
            for i in range(len(X_train)):
                if X_train[i].shape[0] < 12000:
                    new_list = np.zeros([12000 - X_train[i].shape[0], 53])
                    X_train[i] = np.concatenate([X_train[i], new_list], 0)
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            

            loss_tracker = LossComponentsCallback(m)

            # Add early stopping
            early_stopping = EarlyStopping(
                monitor='loss',           
                patience=20,              
                restore_best_weights=True, 
                min_delta=0.001,          
                verbose=1                 
            )

            hist = m.fit(X_train, y_train, epochs = epoch, batch_size=len(X_train),callbacks=[early_stopping,loss_tracker])


            input_shapes = []
            for i in range(len(X_test)):
                input_shapes.append(np.array(X_test[i]).shape[0])
            m.set_input_shapes(input_shapes)

            for i in range(len(X_test)):
                if X_test[i].shape[0] < 12000:
                    new_list = np.zeros([12000 - X_test[i].shape[0], 53])
                    X_test[i] = np.concatenate([X_test[i], new_list], 0)
            X_test = np.array(X_test)
            x_c = copy.deepcopy(X_test)
            y_test = np.array(y_test)
            y_pred_test = m.predict(X_test)
            y_pred_test = np.array(y_pred_test[:,0])

            y_difference = np.mean(np.abs(np.abs(y_test) - np.abs(y_pred_test)))
            eval = m.evaluate(X_test, y_test)
            print("The mean absolute difference between y_tru & y_pred is : {}" .format(str(y_difference)))

            final_train_loss = loss_tracker.total_losses[-1] if loss_tracker.total_losses else None

        # Plot all loss components over epochs
        plt.figure(figsize=(12, 8))

        epoch_length = range(1, len(loss_tracker.total_losses) + 1)

        # Total loss
        plt.subplot(2, 2, 1)
        plt.plot(epoch_length, loss_tracker.total_losses, 'b-', label='Total Loss')
        plt.title('Total Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
end = time.time()           
print("Number of epochs: {} Epoch runtime is {} minutes".format(str(epoch),((end - start)/60)))



---------- Hyperparameter combinations ------------
Epoch : 100;  physics_weight: 2e-06;


2025-04-28 15:50:55.603444: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-04-28 15:50:55.603478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (curie.cluster): /proc/driver/nvidia/version does not exist
2025-04-28 15:50:55.604545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
Inside call
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Inside call
1/1 [==============================] - 134s 134s/step - loss: 50.0718
Epoch 2/100
1/1 [==============================] - 61s 61s/step - loss: 38.6989
Epoch 3/100
1/1 [==============================] - 50s 50s/step - loss: 39.1198
Epoch 4/100
1/1 [==============================] - 50s 50s/step - loss: 38.5468
Epoch 5/100
1/1 [==============================] - 50s 50s/step - loss: 38.3115
Epoch 6/100
1/1 [==============================] - 51s 51s/step - loss: 38.4734
Epoch 7/100
1/1 [==============================] - 50s 50s/step - loss: 38.3098
Epoch 8/100
1/1 [==============================] - 48s 48s/step - loss: 37.9689
Epoch 9/100
1/1 [==============================] - 49s 49s/step - loss: 37.8320
Epoch 10/100
1/1 [==============================] - 5

In [21]:
print(tf.config.list_physical_devices('GPU')) 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)  # Now applies to GPU 3

In [ ]:
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    x = tf.random.normal((10000, 10000))
    y = tf.matmul(x, x)
    print(y.numpy().mean()) 

In [ ]:
%cd BFE-Loss-Function/

In [ ]:
PDBs_BRD = pickle.load(open('PDBs_BRD4.pkl', 'rb'))
df_test = pd.read_csv('BRD4_Test.csv')

In [ ]:
info_brd4 = []
for pdb in list(PDBs_BRD.keys()):
    info_brd4.append(df_test[df_test['Ids'] == pdb][['pb_host_VDWAALS', 'pb_guest_VDWAALS', 'pb_complex_VDWAALS', 'gb_host_1-4EEL', 'gb_guest_1-4EEL', 'gb_Complex_1-4EEL',
       'gb_host_EELEC', 'gb_guest_EELEC', 'gb_Complex_EELEC', 'gb_host_EGB', 'gb_guest_EGB', 'gb_Complex_EGB', 'gb_host_ESURF', 'gb_guest_ESURF', 'gb_Complex_ESURF']].to_numpy()[0])

In [ ]:
df_test

In [ ]:
X_brd4 = []
y_brd4 = []
for i, pdb in enumerate(list(PDBs_BRD.keys())):
    X_brd4.append(featurize(PDBs_BRD[pdb], info_brd4[i]))
    y_brd4.append(df_test[df_test['Ids'] == pdb]['Ex _G_(kcal/mol)'].to_numpy()[0])


In [ ]:
len(X_brd4), len(y_brd4)

In [ ]:
# X_brd4

In [ ]:

input_shapes = []
for i in range(len(X_brd4)):
    input_shapes.append(np.array(X_brd4[i]).shape[0])
m.set_input_shapes(input_shapes)

for i in range(len(X_brd4)):
    if X_brd4[i].shape[0] < 2000:
        new_list = np.zeros([2000 - X_brd4[i].shape[0], 53])
        X_brd4[i] = np.concatenate([X_brd4[i], new_list], 0)
X_brd4 = np.array(X_brd4)
x_c = copy.deepcopy(X_brd4)
y_brd4 = np.array(y_brd4)
y_brd4_test = m.predict(X_brd4)
y_brd4_test = np.array(y_brd4_test[:,0])

y_difference = np.mean(np.abs(y_brd4 - y_brd4_test))
eval = m.evaluate(X_brd4, y_brd4,len(X_brd4))
print("The mean absolute difference between y_tru & y_pred is : {}" .format(str(y_difference)))

print(y_difference)




In [ ]:
# y_pred = m.predict(X_test, batch_size=53)
# print("Predictions:", y_pred)